In [54]:
# Importación de librerías
import pandas as pd
import numpy as np

# Cálculo de stock e impacto económico

Cargamos los ficheros de RMSE de modelos ml y naives.

In [55]:
RMSE_naive = pd.read_csv('Predicciones_Test_Naive_Modelo.csv')
RMSE_ML = pd.read_csv('Mejores_Modelos_por_Producto.csv')

In [56]:
#Ordenamos por producto
RMSE_ML.sort_values(by=['producto'], inplace=True)
RMSE_ML

,producto,modelo,RMSE
0,1,RF,7.592185
1,2,RF,5.349819
2,3,RF,4.348743
3,4,RF,3.780251
4,5,RF,3.307847
...,...,...,...
889,996,MLP,2.108776
890,997,MLP,1.717315
891,998,RF,2.354383
892,999,RF,2.933878


In [58]:
RMSE_naive.sort_values(by=['producto'], inplace=True)
RMSE_naive

,producto,RMSE
0,1,9.570524
1,2,8.206589
2,3,6.502190
3,4,3.691420
4,5,5.332608
...,...,...
889,996,2.393292
890,997,1.891018
891,998,2.631359
892,999,4.141485


Importamos los datos de ciclo de aprovisionamiento

In [59]:
DatosCicloAprovisionamiento = pd.read_excel('DatosCicloAprovisionamiento.xlsx')


# Calculamos la raiz del ciclo de aprovisionamiento
DatosCicloAprovisionamiento['RaizCicloAprovisionamiento'] = np.sqrt(DatosCicloAprovisionamiento['diasEntrePedidos'] * DatosCicloAprovisionamiento['diasLeadtime'])

DatosCicloAprovisionamiento.head()

,producto,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento
0,1,14,15,14.491377
1,2,14,15,14.491377
2,3,14,15,14.491377
3,4,14,15,14.491377
4,5,14,15,14.491377


In [60]:
# Ahora calculamos las unidades de stock
factor_servicio = 1.64  # Para un nivel de servicio del 95%

DatosCicloAprovisionamiento_ML = DatosCicloAprovisionamiento.merge(RMSE_ML, on='producto')
DatosCicloAprovisionamiento_ML['UnidadesStock'] = factor_servicio * DatosCicloAprovisionamiento_ML['RMSE'] * DatosCicloAprovisionamiento_ML['RaizCicloAprovisionamiento']
DatosCicloAprovisionamiento_ML.head()

,producto,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,modelo,RMSE,UnidadesStock
0,1,14,15,14.491377,RF,7.592185,180.434786
1,2,14,15,14.491377,RF,5.349819,127.143028
2,3,14,15,14.491377,RF,4.348743,103.351606
3,4,14,15,14.491377,RF,3.780251,89.840913
4,5,14,15,14.491377,RF,3.307847,78.613829


In [61]:
# Importamos el archivo con los datos de precios
DatosPrecioMedio = pd.read_excel('DatosPrecioMedio.xlsx')
DatosPrecioMedio.head()

,producto,eurPrecioMedio
0,1,68.730000
1,2,148.330000
2,3,169.000000
3,4,0.604383
4,5,4.553314


In [62]:
#Calculamos el coste diario
DatosPrecioMedio_ML = DatosPrecioMedio.merge(DatosCicloAprovisionamiento_ML, on='producto')

factor = 0.05
DatosPrecioMedio_ML['CosteDiarioStock'] = DatosPrecioMedio_ML['UnidadesStock'] * DatosPrecioMedio_ML['eurPrecioMedio'] * factor 

DatosPrecioMedio_ML.head()

,producto,eurPrecioMedio,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,modelo,RMSE,UnidadesStock,CosteDiarioStock
0,1,68.730000,14,15,14.491377,RF,7.592185,180.434786,620.064144
1,2,148.330000,14,15,14.491377,RF,5.349819,127.143028,942.956270
2,3,169.000000,14,15,14.491377,RF,4.348743,103.351606,873.321068
3,4,0.604383,14,15,14.491377,RF,3.780251,89.840913,2.714916
4,5,4.553314,14,15,14.491377,RF,3.307847,78.613829,17.897673


In [63]:
# Calculo del coste anual

DatosPrecioMedio_ML['CosteAnualStock'] = DatosPrecioMedio_ML['CosteDiarioStock'] * 365
DatosPrecioMedio_ML.head()

,producto,eurPrecioMedio,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,modelo,RMSE,UnidadesStock,CosteDiarioStock,CosteAnualStock
0,1,68.730000,14,15,14.491377,RF,7.592185,180.434786,620.064144,226323.412415
1,2,148.330000,14,15,14.491377,RF,5.349819,127.143028,942.956270,344179.038566
2,3,169.000000,14,15,14.491377,RF,4.348743,103.351606,873.321068,318762.189756
3,4,0.604383,14,15,14.491377,RF,3.780251,89.840913,2.714916,990.944258
4,5,4.553314,14,15,14.491377,RF,3.307847,78.613829,17.897673,6532.650803


In [64]:
# Suma de los costes de una año

sum_coste_anual_ML = DatosPrecioMedio_ML['CosteAnualStock'].sum()
print(f'El coste anual total de stock es: {sum_coste_anual_ML} euros')

El coste anual total de stock es: 24118325.48563359 euros


Hacemos lo mismo con el modelo Naive

In [65]:
# Calculamos las unidades de stock
factor_servicio = 1.64  # Para un nivel de servicio del 95%

DatosCicloAprovisionamiento_NF = DatosCicloAprovisionamiento.merge(RMSE_naive, on='producto')
DatosCicloAprovisionamiento_NF['UnidadesStock'] = factor_servicio * DatosCicloAprovisionamiento_NF['RMSE'] * DatosCicloAprovisionamiento_NF['RaizCicloAprovisionamiento']
DatosCicloAprovisionamiento_NF.head()

,producto,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,RMSE,UnidadesStock
0,1,14,15,14.491377,9.570524,227.451722
1,2,14,15,14.491377,8.206589,195.036625
2,3,14,15,14.491377,6.502190,154.530135
3,4,14,15,14.491377,3.691420,87.729765
4,5,14,15,14.491377,5.332608,126.734006


In [66]:
#Calculamos el coste diario
DatosPrecioMedio_NF = DatosPrecioMedio.merge(DatosCicloAprovisionamiento_NF, on='producto')

factor = 0.05
DatosPrecioMedio_NF['CosteDiarioStock'] = DatosPrecioMedio_NF['UnidadesStock'] * DatosPrecioMedio_NF['eurPrecioMedio'] * factor 

DatosPrecioMedio_NF.head()

,producto,eurPrecioMedio,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,RMSE,UnidadesStock,CosteDiarioStock
0,1,68.730000,14,15,14.491377,9.570524,227.451722,781.637842
1,2,148.330000,14,15,14.491377,8.206589,195.036625,1446.489131
2,3,169.000000,14,15,14.491377,6.502190,154.530135,1305.779638
3,4,0.604383,14,15,14.491377,3.691420,87.729765,2.651119
4,5,4.553314,14,15,14.491377,5.332608,126.734006,28.852988


In [67]:
# Calculo del coste anual

DatosPrecioMedio_NF['CosteAnualStock'] = DatosPrecioMedio_NF['CosteDiarioStock'] * 365
DatosPrecioMedio_NF.head()

,producto,eurPrecioMedio,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,RMSE,UnidadesStock,CosteDiarioStock,CosteAnualStock
0,1,68.730000,14,15,14.491377,9.570524,227.451722,781.637842,285297.812379
1,2,148.330000,14,15,14.491377,8.206589,195.036625,1446.489131,527968.532707
2,3,169.000000,14,15,14.491377,6.502190,154.530135,1305.779638,476609.567961
3,4,0.604383,14,15,14.491377,3.691420,87.729765,2.651119,967.658326
4,5,4.553314,14,15,14.491377,5.332608,126.734006,28.852988,10531.340579


In [68]:
# Suma de los costes de una año

sum_coste_anual_NF = DatosPrecioMedio_NF['CosteAnualStock'].sum()
print(f'El coste anual total de stock es: {sum_coste_anual_NF} euros')

El coste anual total de stock es: 33892712.41255322 euros
